In [ ]:
#code below works fine dont change

In [3]:
import cv2
import dlib
import face_recognition
import numpy as np
import pymysql

# Connect to the MySQL database
db = pymysql.connect(host='localhost', user='root', password='52456', database='mydatabase')
cursor = db.cursor()

def load_reference_embedding(cursor, account_number):
    # Load the reference face embedding from the database for the given account number
    query = "SELECT face_embedding FROM face_db WHERE account_number = %s"
    cursor.execute(query, (account_number,))
    result = cursor.fetchone()

    if result:
        embedding_str = result[0]
        embedding = np.fromstring(embedding_str, sep=' ')
        return embedding.reshape(-1, 128)
    else:
        return None


def compare_face_embeddings(face_embedding, reference_embedding):
    # Compute the distance between the face embeddings
    if reference_embedding is not None:
        distance = np.linalg.norm(face_embedding - reference_embedding)
        if distance < 0.6:
            return True
    return False


def face_verification(cursor, connection, account_number):
    # Start the webcam for capturing images
    video_capture = cv2.VideoCapture(0)

    while True:
        # Capture frame-by-frame
        ret, frame = video_capture.read()

        # Convert the image from BGR color (OpenCV default) to RGB color
        rgb_image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        # Detect faces in the frame
        face_locations = face_recognition.face_locations(rgb_image)

        # If no face is detected, continue to the next frame
        if len(face_locations) == 0:
            cv2.imshow('Video', frame)
            cv2.waitKey(1)
            continue

        # Get the face embedding of the detected face
        face_encoding = face_recognition.face_encodings(rgb_image, face_locations)[0]

        # Retrieve the reference embedding for the account number
        reference_embedding = load_reference_embedding(cursor, account_number)

        # Compare the face embedding with the reference embedding
        verification_result = compare_face_embeddings(face_encoding, reference_embedding)

        # Display the captured image with bounding boxes
        for (top, right, bottom, left) in face_locations:
            cv2.rectangle(frame, (left, top), (right, bottom), (0, 255, 0), 2)

        if verification_result:
            cv2.putText(frame, "Verification Successful", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
        else:
            cv2.putText(frame, "Verification Failed", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

        cv2.imshow('Video', frame)

        # Press 'q' to quit the program
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
        
        video_capture.release()

    # Release the webcam and close the OpenCV windows
    video_capture.release()
    
    cv2.destroyAllWindows()
    connection.close()


In [4]:
# Example usage: Perform face verification for account number 123456789
account_number = 129098788
face_verification(cursor, db, account_number)

# Close the database connection
db.close()

/tmp/ipykernel_17365/371490467.py:19: DeprecationWarning: string or file could not be read to its end due to unmatched data; this will raise a ValueError in the future.
  embedding = np.fromstring(embedding_str, sep=' ')


error: OpenCV(4.7.0) /io/opencv/modules/imgproc/src/color.cpp:182: error: (-215:Assertion failed) !_src.empty() in function 'cvtColor'


In [1]:
import cv2
import dlib
import face_recognition
import numpy as np
import pymysql

# Connect to the MySQL database
db = pymysql.connect(host='localhost', user='root', password='52456', database='mydatabase')
cursor = db.cursor()

def load_reference_embedding(cursor, account_number):
    # Load the reference face embedding from the database for the given account number
    query = "SELECT face_embedding FROM face_db WHERE account_number = %s"
    cursor.execute(query, (account_number,))
    result = cursor.fetchone()

    if result:
        embedding_bytes = result[0]
        embedding = np.frombuffer(embedding_bytes, dtype=np.float64)
        return embedding.reshape(-1, 128)
    else:
        return None


def compare_face_embeddings(face_embedding, reference_embedding):
    # Compute the distance between the face embeddings
    if reference_embedding is not None:
        distance = np.linalg.norm(face_embedding - reference_embedding)
        if distance < 0.6:
            return True
    return False


def face_verification(cursor, connection, account_number):
    # Start the webcam for capturing images
    video_capture = cv2.VideoCapture(0)

    while True:
        # Capture frame-by-frame
        ret, frame = video_capture.read()

        # Convert the image from BGR color (OpenCV default) to RGB color
        rgb_image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        # Detect faces in the frame
        face_locations = face_recognition.face_locations(rgb_image)

        # If no face is detected, continue to the next frame
        if len(face_locations) == 0:
            cv2.imshow('Video', frame)
            cv2.waitKey(1)
            continue

        # Get the face embedding of the detected face
        face_encoding = face_recognition.face_encodings(rgb_image, face_locations)[0]

        # Retrieve the reference embedding for the account number
        reference_embedding = load_reference_embedding(cursor, account_number)

        # Compare the face embedding with the reference embedding
        verification_result = compare_face_embeddings(face_encoding, reference_embedding)

        # Display the captured image with bounding boxes
        for (top, right, bottom, left) in face_locations:
            cv2.rectangle(frame, (left, top), (right, bottom), (0, 255, 0), 2)

        if verification_result:
            cv2.putText(frame, "Verification Successful", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
            
        else:
            cv2.putText(frame, "Verification Failed", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

        cv2.imshow('Video', frame)

        # Press 'q' to quit the program
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
        video_capture.release()   
    # Release the webcam and close the OpenCV windows
    video_capture.release()
    cv2.destroyAllWindows()
    connection.close()

# Example usage: Perform face verification for account number 123456789
account_number = '129098788'
face_verification(cursor, db, account_number)

# Close the database connection
db.close()


ValueError: buffer size must be a multiple of element size